In [1]:
import numpy as np
import pandas as pd

from sklearn import (
    model_selection, tree, metrics, svm
)

## loading data

In [2]:
X = 'AimoScore_WeakLink_big_scores (1).xls'
Y = '20190108 scores_and_weak_links.xlsx'

X = pd.read_excel(X, index_col= 'ID')
Y = pd.read_excel(Y, index_col= 'ID').drop(['Date','SCORE'], axis= 1).T.apply(np.argmax).rename('weakest link')

df = X.join(Y)
df

c:\Users\hahme\miniconda3\envs\DIS\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,AimoScore,No_1_Angle_Deviation,No_2_Angle_Deviation,No_3_Angle_Deviation,No_4_Angle_Deviation,No_5_Angle_Deviation,No_6_Angle_Deviation,No_7_Angle_Deviation,No_8_Angle_Deviation,No_9_Angle_Deviation,...,No_20_NASM_Deviation,No_21_NASM_Deviation,No_22_NASM_Deviation,No_23_NASM_Deviation,No_24_NASM_Deviation,No_25_NASM_Deviation,No_1_Time_Deviation,No_2_Time_Deviation,EstimatedScore,weakest link
ID,,,,,,,,,,,,,,,,,,,,,
0003cdcc-86ed-494a-a3b5-90d09e96e06b.Kinect,0.323667,0.538020,0.815878,0.346724,0.382114,0.302248,0.947872,0.275945,0.521760,0.457198,...,0.656624,0.642276,0.552846,0.648972,0.578192,0.560019,0.821616,0.818747,0.209947,4
003115c4-bdb8-491c-b571-8fcebdecf8ed.Kinect,0.323699,0.443807,0.306552,0.823529,0.188905,0.497370,0.140124,0.664275,0.521760,0.729316,...,0.721186,0.826399,0.805356,0.848876,0.889048,0.816834,0.307987,0.248207,0.457198,2
00316bfb-ed43-489f-a55b-11c7f01c852d.Kinect,0.848327,0.603539,0.373984,0.346724,0.590626,0.341942,0.298900,0.276901,0.623625,0.658058,...,0.656624,0.642276,0.690579,0.648972,0.578192,0.555715,0.218556,0.235294,0.107126,1
00607608-6f2f-459b-a69d-e14067489459.Kinect,0.351332,0.484935,0.623625,0.380201,0.975132,0.509326,0.888570,0.363462,0.847441,0.237207,...,0.656624,0.642276,0.552846,0.648972,0.578192,0.744620,0.458154,0.432807,0.612626,4
007396ec-3463-4a05-915c-02244ff8d3de.Kinect,0.627181,0.860832,0.657580,0.745576,0.552846,0.375897,0.483022,0.388331,0.521760,0.387853,...,0.656624,0.642276,0.552846,0.648972,0.578192,0.308943,0.805356,0.774271,0.153515,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ff015ddd-c9fb-49e3-8e33-abbf72e8b120.Kinect,0.527640,0.404489,0.214422,0.822827,0.937440,0.739255,0.909265,0.681471,0.658071,0.822350,...,0.886819,0.640401,0.525310,0.776982,0.822827,0.367717,0.584050,0.570201,0.613658,1
ff514cba-6c5e-41a9-9936-3e8400565c8d.Kinect,0.407197,0.279770,0.131038,0.346724,0.317073,0.945481,0.646581,0.913439,0.521760,0.570540,...,0.945002,0.642276,0.596365,0.648972,0.749880,0.943568,0.684840,0.711621,0.877571,11
ff514cba-6c5e-41a9-9936-3e8400565c8d.Kinect,0.407197,0.606972,0.863897,0.467526,0.225406,0.946036,0.706781,0.934097,0.303247,0.576409,...,0.886819,0.640401,0.725883,0.757402,0.822827,0.943649,0.953200,0.956543,0.761700,11


## applying the train test split

In [3]:
X  = df.drop(['AimoScore','EstimatedScore', 'weakest link'], axis= 1)
Y1 = df['AimoScore']
Y2 = df['weakest link']

trainX, testX, trainY1, testY1, trainY2, testY2 = model_selection.train_test_split(X, Y1, Y2, test_size= 0.2)

# classification with svm

## grid search optimization

In [4]:
model = model_selection.GridSearchCV(
    estimator = svm.SVC(),
    param_grid= {
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 
        'degree': [1,2,3,4,5],
        'gamma' : ['scale','auto'],
        'C'     : [1, 10],
    },
)
model.fit(trainX,trainY2)

testP2 = model.predict(testX)
metrics.accuracy_score(testY2,testP2)

c:\Users\hahme\miniconda3\envs\DIS\Lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


0.6062052505966588